In [1]:
import os
import time
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2
print(pd.__version__)

from postgressql import *

1.3.3


In [16]:
conn_params_dic = {
    "host": "10.0.0.102",
    "dbname": "mastr",
    "user": "uba_user",
    "password": "UBAit2021!",
    "port": "5432"
}

sqlpath = "./sqlCommands/loadMastrDBfiles.sql"

def build_postgres_conn_string (param:dict) -> str:
    """

    """
    return f'postgresql+psycopg2://{param["user"]}:{param["password"]}@{param["host"]}:{param["port"]}/{param["dbname"]}'

def create_postgres_engine (param:dict):
    """
    
    """
    conString = build_postgres_conn_string(conn_params_dic)
    engine = create_engine(conString, pool_recycle=3600)
    return engine

def read_postgres_from_queryfile (sqlpath:str, postgresLogin:dict) -> pandas.core.frame.DataFrame:
    """

    """
    engine = create_postgres_engine(postgresLogin)
    
    scriptFile = open(sqlpath,'r')
    script = scriptFile.read()
    df = pd.read_sql(script, engine)

    return df

# CREATE TABLE based on Pandas DF

In [55]:
import numpy as np

filepath = "/uba/mastr/MaStR/Vollauszüge/"
dateDirs = os.listdir(filepath)
array = np.array(dateDirs).astype("int")
recentDir = str(array.min())
filepath = filepath + recentDir + "/"
print(filepath)

/uba/mastr/MaStR/Vollauszüge/20211004/


In [56]:
filepath = "/uba/mastr/MaStR/Vollauszüge/"
dateDirs = os.listdir(filepath)
array = np.array(dateDirs).astype("int")
recentDir = str(array.min())
filepath = filepath + recentDir + "/"
files = os.listdir(filepath)
files.sort()
filenames = [element.split(".")[0].split("_")[0] for element in files]
uniqueFilenames = list(set(filenames))

filterString = 'Wind'
filteredList = [k for k in files if filterString in k]
selectedTable = filteredList[1].split(".")[0]
xmlPath = filepath + selectedTable + ".xml"
print(xmlPath)

df = from_xml_to_DataFrame(xmlPath)
change_Dtype_Datetime(df)
df.head()

/uba/mastr/MaStR/Vollauszüge/20211004/EinheitenWind.xml


,EinheitMastrNummer,DatumLetzteAktualisierung,LokationMaStRNummer,NetzbetreiberpruefungStatus,NetzbetreiberpruefungDatum,AnlagenbetreiberMastrNummer,Land,Bundesland,Landkreis,Gemeinde,...,Kuestenentfernung,GeplantesInbetriebnahmedatum,Strasse,Hausnummer,Adresszusatz,DatumEndgueltigeStilllegung,ClusterOstsee,DatumBeginnVoruebergehendeStilllegung,Kraftwerksnummer,DatumWiederaufnahmeBetrieb
0,SEE900002935310,2021-08-20 09:51:42.953594100,SEL957516775208,0,2021-08-20,ABR935787334270,84,1400.0,Oder-Spree,Jacobsdorf,...,NaN,NaT,None,None,None,NaT,NaN,NaT,None,NaT
1,SEE900004001082,2021-01-27 20:20:59.429862800,SEL949397946808,1,2021-01-27,ABR967616635029,84,1409.0,Paderborn,Büren,...,NaN,NaT,None,None,None,NaT,NaN,NaT,None,NaT
2,SEE900011187731,2020-01-08 18:07:22.967045100,SEL901928237038,1,2020-01-10,ABR962772458710,84,1410.0,Eifelkreis Bitburg-Prüm,Hütten,...,NaN,NaT,None,None,None,NaT,NaN,NaT,None,NaT
3,SEE900013535608,2021-01-21 15:14:32.282210000,SEL956642007177,1,2020-10-29,ABR955714105537,84,1410.0,Donnersbergkreis,Gauersheim,...,NaN,NaT,None,None,None,NaT,NaN,NaT,None,NaT
4,SEE900014803461,2021-03-25 13:33:47.532137200,SEL977165118843,0,2021-03-25,ABR918453648000,84,1415.0,Erfurt,Erfurt,...,NaN,NaT,None,None,None,NaT,NaN,NaT,None,NaT


In [13]:
selectedTable

'EinheitenWind'

In [15]:
engine = create_postgres_engine(conn_params_dic)
df.to_sql(
    name=selectedTable,
    schema="mastr_raw",
    con=engine,
    if_exists="replace",
    index=False
)

# USING PANDAS AND SQLALCHEMY TO READ 

In [17]:
df = read_postgres_from_queryfile(sqlpath, conn_params_dic)
df.head()

,index,EinheitMastrNummer,DatumLetzteAktualisierung,LokationMaStRNummer,NetzbetreiberpruefungStatus,NetzbetreiberpruefungDatum,AnlagenbetreiberMastrNummer,Land,Bundesland,Landkreis,...,Kuestenentfernung,GeplantesInbetriebnahmedatum,Strasse,Hausnummer,Adresszusatz,DatumEndgueltigeStilllegung,ClusterOstsee,DatumBeginnVoruebergehendeStilllegung,Kraftwerksnummer,DatumWiederaufnahmeBetrieb
0,0,SEE900002935310,2021-08-20 09:51:42.953594,SEL957516775208,0,2021-08-20,ABR935787334270,84,1400.0,Oder-Spree,...,NaN,NaT,None,None,None,NaT,NaN,NaT,None,NaT
1,1,SEE900004001082,2021-01-27 20:20:59.429862,SEL949397946808,1,2021-01-27,ABR967616635029,84,1409.0,Paderborn,...,NaN,NaT,None,None,None,NaT,NaN,NaT,None,NaT
2,2,SEE900011187731,2020-01-08 18:07:22.967045,SEL901928237038,1,2020-01-10,ABR962772458710,84,1410.0,Eifelkreis Bitburg-Prüm,...,NaN,NaT,None,None,None,NaT,NaN,NaT,None,NaT
3,3,SEE900013535608,2021-01-21 15:14:32.282210,SEL956642007177,1,2020-10-29,ABR955714105537,84,1410.0,Donnersbergkreis,...,NaN,NaT,None,None,None,NaT,NaN,NaT,None,NaT
4,4,SEE900014803461,2021-03-25 13:33:47.532137,SEL977165118843,0,2021-03-25,ABR918453648000,84,1415.0,Erfurt,...,NaN,NaT,None,None,None,NaT,NaN,NaT,None,NaT


# USING PSYCOPG2

In [ ]:
conn_params_dic = {
    "host": "10.0.0.102",
    "dbname": "mastr",
    "user": "uba_user",
    "password": "UBAit2021!",
    "port": "5432"
}
conn = connect(conn_params_dic)
print(type(conn))

copy_from_DataFrame()